In [46]:
%matplotlib inline

from eqcart import Cartogram
from chorogrid import Chorogrid
import geopandas as gpd
import json
import numpy as np
import seaborn as sns
import streamlit as st
import pandas as pd
import pydeck as pdk
import random
from cartutils import is_valid, delete_old_point, update_new_point, shunt_point


## DataFrame Choices
geo_data - GeoJson of the UK for Plotly with the dummy colors of the results

hex_data - Hexified UK Constituencies map with randomized colors of the results

In [21]:
##geo_data is a dataframe with the centroids of the polygons and overlapping
#downloaded from https://martinjc.github.io/UK-GeoJSON/ *ENGLAND > National > \
# Local Authority Districts and converted to GEOJson

#Open the JSON file and load it into a variable
with open("json_files/England.geojson", "r") as file:
    geo_data = json.load(file)

#Changing the centroid lons and lats as a DataFrame
gdf = gpd.GeoDataFrame.from_features(geo_data["features"])
coordinates = gdf.get_coordinates()
coordinates.columns = ["lng", "lat"]
centroids = gdf.centroid.to_frame()
geo_data = pd.DataFrame()
geo_data["lng"] = gdf.centroid.x
geo_data["lat"] = gdf.centroid.y

#setting results as random integers between 0 to 3
geo_data['fake_results'] = np.random.randint(0, 4, geo_data.shape[0])


In [43]:
geo_data['id'] = geo_data.index
geo_data.rename(columns={'lng': 'longitude','lat':'latitude'}, inplace=True)


In [45]:

geo_data.to_csv('geo_data.csv', index=False)


## The current working set using geo_data and pydeck
Current problems include:
- hexagons not side by side
- map still appearing when running the py file but not in Jupyter

In [22]:
#styling the map f hexes
# map colors
color_scale = {
    0: [255, 0, 0, 1],  # Red
    1: [0, 0, 255, 1],  # Blue
    2: [128, 128, 128, 1],  # Grey
    3: [0, 255, 0, 1],  # Green
}

# Map "fake_results" values to RGB colors
geo_data["color"] = geo_data["fake_results"].map(color_scale)

layer = pdk.Layer(
    "HexagonLayer",
    geo_data,
    get_position=["lng", "lat"],
    elevation_scale=0,
    pickable=True,
    elevation_range=[0, 3000],
    filled=True,
    coverage=10,
    get_fill_color="color",  # Use the new "color" column
)

view_state = pdk.ViewState(
    longitude=-1.415,
    latitude=52.2323,
    zoom=5,
    min_zoom=5,
    max_zoom=15,
    pitch=0,
    bearing=0,
)

# Render without the map
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    map_style="",
    map_provider="",
    api_keys=None,
)


In [23]:
# Streamlit app layout with sliders and titles
st.title("UK, hun?")

# Create a placeholder for the map on the left side
st.sidebar.header("Interactive Hex Map")

# Create sliders for conservative and labor party approval ratings on the right side
st.sidebar.header("Sliders for Polling/Approval Ratings")

conservative_rating = st.sidebar.slider(
    "Conservative Rating", min_value=0, max_value=100, value=50
)
labor_party_rating = st.sidebar.slider(
    "Labor Party Rating", min_value=0, max_value=100, value=50
)

# Display the current values of the sliders
st.sidebar.text(f"Current Conservative Rating: {conservative_rating}%")
st.sidebar.text(f"Current Labor Party Rating: {labor_party_rating}%")

# Display the PyDeck chart using st.pydeck_chart
st.pydeck_chart(r)


2023-11-30 11:51:13.468 
  command:

    streamlit run /home/asia/.pyenv/versions/3.10.6/envs/UK_election/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

## Testing a new layer style

In [24]:
#new layerstyle using h3_hex
h3_layer = pdk.Layer(
    "H3HexagonLayer",
    geo_data,
    pickable=True,
    stroked=True,
    filled=True,
    extruded=False,
    get_hexagon="hex",
    get_fill_color="[255 - count, 255, count]",
    get_line_color=[255, 255, 255],
    line_width_min_pixels=2,
)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=-1.415,
    latitude=52.2323,
    zoom=5,
    min_zoom=5,
    max_zoom=15,
    pitch=0,
    bearing=0,
)

# Render
r_new = pdk.Deck(layers=[h3_layer], initial_view_state=view_state, tooltip={"text": "Count: {count}"})
r_new.to_html("h3_hexagon_layer.html")


In [11]:

# Streamlit app layout
st.title("UK, hun?")

# Create a placeholder for the map on the left side
st.sidebar.header("Interactive Hex Map")

# Create sliders for conservative and labor party approval ratings on the right side
st.sidebar.header("Sliders for Polling/Approval Ratings")
conservative_rating = st.sidebar.slider(
    "Conservative Rating", min_value=0, max_value=100, value=50
)
labor_party_rating = st.sidebar.slider(
    "Labor Party Rating", min_value=0, max_value=100, value=50
)

# Display the current values of the sliders
st.sidebar.text(f"Current Conservative Rating: {conservative_rating}%")
st.sidebar.text(f"Current Labor Party Rating: {labor_party_rating}%")

# Display the PyDeck chart using st.pydeck_chart
st.pydeck_chart(r_new)


DeltaGenerator()

## Trying the Hexbins with Plotly layer styles instead from 
https://plotly.com/python/hexbin-mapbox/

In [25]:
hex_data = "json_files/uk-constituencies-2023.json"


In [26]:
hex_df = pd.read_json(hex_data)
hex_df['hexes'][1]


/tmp/ipykernel_16269/2072595757.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



{'n': 'Aldridge-Brownhills',
 'r': -30,
 'q': 56,
 'region': 'E12000005',
 'colour': '#E6007C'}

In [27]:
#apply hexes, apply id
hex_values = hex_df['hexes'].apply(pd.Series)
hex_values['randNumCol'] = np.random.randint(0, 4, hex_values.shape[0])


In [28]:
#add color values
hex_values['colour'] = hex_values['randNumCol'].map({
    0: '#cd5c5c',   # Indian Red
    1: '#367588',   # Blue
    2: '#49796b',   # Hooker Green
    3: '#d3d3d3'    # Light Gray
})


In [29]:
#reset index name and id
hex_values.index.name = 'id'
hex_values['id'] = hex_values.index
hex_values.reset_index(drop=True, inplace=True)


In [30]:
hex_values.to_csv('hex_values.csv', index=False)

hex_values.head()


,n,r,q,region,colour,randNumCol,id
0,Aldershot,-40,56,E12000008,#d3d3d3,3,E14001063
1,Aldridge-Brownhills,-30,56,E12000005,#49796b,2,E14001064
2,Altrincham and Sale West,-25,52,E12000002,#d3d3d3,3,E14001065
3,Amber Valley,-27,58,E12000004,#367588,1,E14001066
4,Arundel and South Downs,-44,61,E12000008,#49796b,2,E14001067


In [31]:
import plotly.figure_factory as ff
import plotly.express as px

# token = pk.eyJ1Ijoibm5vYmxlMTMiLCJhIjoiY2xwam91bHNoMDF4dTJpbGIzaTRwMWdubSJ9.t9AstZVQrSa9uFByagdtNw

# # px.set_mapbox_access_token(open(token).read())
# px.set_mapbox_access_token(token)
df = hex_values


In [37]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   n           650 non-null    object
 1   r           650 non-null    int64 
 2   q           650 non-null    int64 
 3   region      650 non-null    object
 4   colour      650 non-null    object
 5   randNumCol  650 non-null    int64 
 6   id          650 non-null    object
dtypes: int64(3), object(4)
memory usage: 35.7+ KB


In [35]:
# create_hexbin_mapbox(data_frame, lat, lon, color, nx_hexagon, agg_func, \
# animation_frame, color_discrete_sequence, color_discrete_map, labels, \
# color_continuous_scale, range_color, color_continuous_midpoint, \
# opacity, zoom, center, mapbox_style, title, template, width, \
# height, min_count, show_original_data, original_data_marker)

fig = ff.create_hexbin_mapbox(
    data_frame=df, lat="r", lon="q",color="randNumCol", #color_discrete_map =["red", "blue", "grey"],
    nx_hexagon=22, opacity=0.5,
    min_count=1,
)
fig.show()


In [17]:
with open("json_files/England.geojson", "r") as file:
    test_df = json.load(file)


In [18]:
gdf = gpd.GeoDataFrame.from_features(test_df["features"])
coordinates = gdf.get_coordinates()
coordinates.columns = ["lng", "lat"]
centroids = gdf.centroid.to_frame()
test_df = pd.DataFrame()
test_df["lng"] = gdf.centroid.x
test_df["lat"] = gdf.centroid.y


In [20]:

fig = ff.create_hexbin_mapbox(
    data_frame=test_df, lat="lat", lon="lng", #color="randNumCol", #color_discrete_map =["red", "blue", "grey"],
    nx_hexagon=20, opacity=0.5, labels={"color": "color label"},
    min_count=1,
)
fig.show()


## Testing a fake dataset on Tilegram and Hex Builder

In [13]:
#
hex_values.rename(columns={'region name': 'n'}, inplace=True)
hex_values['colour'] = hex_values['randNumCol'].map({
    0: '#cd5c5c',   # Indian Red
    1: '#367588',   # Blue
    2: '#49796b',   # Hooker Green
    3: '#d3d3d3'    # Light Gray
})


In [60]:
hex_values.rename(columns={'region': 'n'}, inplace=True)


In [15]:
hex_json = hex_values.to_json(orient='records')
hex_json


'[{"n":"Aldershot","r":-40,"q":56,"region":"E12000008","colour":"#d3d3d3","randNumCol":3,"id":"E14001063"},{"n":"Aldridge-Brownhills","r":-30,"q":56,"region":"E12000005","colour":"#cd5c5c","randNumCol":0,"id":"E14001064"},{"n":"Altrincham and Sale West","r":-25,"q":52,"region":"E12000002","colour":"#cd5c5c","randNumCol":0,"id":"E14001065"},{"n":"Amber Valley","r":-27,"q":58,"region":"E12000004","colour":"#49796b","randNumCol":2,"id":"E14001066"},{"n":"Arundel and South Downs","r":-44,"q":61,"region":"E12000008","colour":"#cd5c5c","randNumCol":0,"id":"E14001067"},{"n":"Ashfield","r":-27,"q":60,"region":"E12000004","colour":"#cd5c5c","randNumCol":0,"id":"E14001068"},{"n":"Ashford","r":-42,"q":72,"region":"E12000008","colour":"#367588","randNumCol":1,"id":"E14001069"},{"n":"Ashton-under-Lyne","r":-23,"q":54,"region":"E12000002","colour":"#d3d3d3","randNumCol":3,"id":"E14001070"},{"n":"Aylesbury","r":-35,"q":60,"region":"E12000008","colour":"#49796b","randNumCol":2,"id":"E14001071"},{"n":"

In [16]:
file_path = 'hex.json'

# Open the file in write mode and write the JSON string
with open(file_path, 'w') as json_file:
    json_file.write(hex_json)

print(f"JSON data has been saved to {file_path}")


JSON data has been saved to hex.json


## Trying out the new hexmap

In [47]:
#First import utils
from cartutils import is_valid, delete_old_point, update_new_point, shunt_point

%matplotlib inline

from chorogrid import Chorogrid
import pandas as pd
import geopandas as gpd #not needed if loading from csv or excel
import seaborn as sns


### Step 0
Join the geo lats and lons with the region ID

In [53]:
elec_data = pd.read_csv('data/cleaned_elections_data_with_incumbent.csv')
elec_data_2019 = elec_data[elec_data['year'] == 2019]
elec_data_2019.to_csv('data/elec_data_2019.csv', index=False)


In [65]:
elec_data_2019 = pd.read_csv('data/elec_data_2019.csv')
elec_data_2019.rename(columns={'constituency_id': 'id'}, inplace=True)


,year,id,constituency,country,electorate,total_votes,tornout,conservative_votes,conservative_vote_share,labour_votes,...,main_parties_vote_share,other_parties_votes,other_parties_vote_share,winning_party,conservative_vote_share_previous,labour_vote_share_previous,liberal_democrats_vote_share_previous,other_parties_vote_share_previous,incumbent_party,winning_party_changed
0,2019,W07000049,ABERAVON,Wales,50750,31598,0.622621,6518.0,0.206279,17008.0,...,0.778467,7000.0,0.221533,labour,0.177378,0.681195,0.018005,0.123422,labour,False
1,2019,W07000058,ABERCONWY,Wales,44699,31865,0.712879,14687.0,0.460913,12653.0,...,0.915142,2704.0,0.084858,conservative,0.445941,0.426190,0.029269,0.098600,conservative,False
2,2019,S14000001,ABERDEEN NORTH,Scotland,62489,37413,0.598713,7535.0,0.201401,4939.0,...,0.409483,22093.0,0.590517,other_parties,0.226923,0.300106,0.046059,0.426912,other_parties,False
3,2019,S14000002,ABERDEEN SOUTH,Scotland,65719,45638,0.694441,16398.0,0.359306,3834.0,...,0.553267,20388.0,0.446733,other_parties,0.421325,0.205493,0.058661,0.314521,conservative,True
4,2019,S14000003,AIRDRIE AND SHOTTS,Scotland,64011,39772,0.621331,7011.0,0.176280,12728.0,...,0.531982,18614.0,0.468018,other_parties,0.231909,0.370928,0.021104,0.376059,other_parties,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,2019,E14001059,WYTHENSHAWE AND SALE EAST,England,76313,44759,0.586519,13459.0,0.300699,23855.0,...,0.903170,4334.0,0.096830,labour,0.296231,0.622192,0.032805,0.048772,labour,False
646,2019,E14001060,YEOVIL,England,82468,59260,0.718582,34588.0,0.583665,3761.0,...,0.957746,2504.0,0.042254,conservative,0.544896,0.124874,0.297051,0.033180,conservative,False
647,2019,W07000041,YNYS MON,Wales,51925,36552,0.703938,12959.0,0.354536,10991.0,...,0.655231,12602.0,0.344769,conservative,0.277892,0.418631,0.012819,0.290658,labour,True
648,2019,E14001061,YORK CENTRAL,England,74899,49505,0.660957,13767.0,0.278093,27312.0,...,0.913605,4277.0,0.086395,labour,0.301744,0.651635,0.046621,0.000000,labour,False


In [59]:
england_geo = gpd.read_file('data/England.geojson')
england_geo.shape[0]


533

In [71]:
england_geo.keys()


Index(['id', 'PCON13CD', 'PCON13CDO', 'PCON13NM', 'geometry'], dtype='object')

In [69]:
eng_geo_df = pd.DataFrame.from_dict(pd.json_normalize(england_geo), orient='columns')


In [70]:
eng_geo_df


""
0
1
2
3
4


In [66]:
merged_data = pd.merge(england_geo, elec_data_2019, how='left', on='id')


In [72]:
merged_data.head()


,id,PCON13CD,PCON13CDO,PCON13NM,geometry,year,constituency,country,electorate,total_votes,...,main_parties_vote_share,other_parties_votes,other_parties_vote_share,winning_party,conservative_vote_share_previous,labour_vote_share_previous,liberal_democrats_vote_share_previous,other_parties_vote_share_previous,incumbent_party,winning_party_changed
0,E14000530,E14000530,A01,Aldershot,"POLYGON ((-0.80469 51.24503, -0.80632 51.25335...",2019,ALDERSHOT,England,72617,47932,...,0.963490,1750.0,0.036510,conservative,0.550562,0.316180,0.074300,0.058958,conservative,False
1,E14000531,E14000531,A02,Aldridge-Brownhills,"POLYGON ((-1.99112 52.63661, -1.98866 52.63899...",2019,ALDRIDGE-BROWNHILLS,England,60138,39342,...,0.971862,1107.0,0.028138,conservative,0.654082,0.298496,0.033379,0.014043,conservative,False
2,E14000532,E14000532,A03,Altrincham and Sale West,"POLYGON ((-2.31432 53.35737, -2.31432 53.35856...",2019,ALTRINCHAM AND SALE WEST,England,73107,54763,...,0.959023,2244.0,0.040977,conservative,0.510191,0.388464,0.076738,0.024607,conservative,False
3,E14000533,E14000533,A04,Amber Valley,"POLYGON ((-1.30790 53.00413, -1.30708 53.00353...",2019,AMBER VALLEY,England,69976,45567,...,0.969539,1388.0,0.030461,conservative,0.565476,0.384296,0.024012,0.026216,conservative,False
4,E14000534,E14000534,A05,Arundel and South Downs,"MULTIPOLYGON (((-0.53958 50.86621, -0.53713 50...",2019,ARUNDEL AND SOUTH DOWNS,England,81726,61408,...,0.949925,3075.0,0.050075,conservative,0.623556,0.227197,0.079378,0.069869,conservative,False
